# Lab 10 - Pruning Lakes

Recall that one of the files (starts with `mces`) contains water quality measurements for lakes in the Twin Cities.  In this lab, we will narrow down the list of lakes for which we have at least one of each measurement type (phosphorus and secchi depth) for each year between 2004 and 2015.

## Tasks

Build a query that leads to a list of lake names and codes that fit the following criteria.

1. Only contains years after 2003.
2. Only contains lakes that have at least one non-null measurement of each type in each year.
3. Contains both the lake name and the lake code.


## Suggested workflow

1. filter and mutate as needed.
2. group and aggregate (hint: You will need to do this twice).
3. filter on the number of observations per year (we want 11, one for each year between 2004-2014).

In [11]:
import pandas as pd
date_cols =['START_DATE']
mces_lakes = (pd.read_csv('./MinneMUDAC_raw_files/mces_lakes_1999_2014_update.csv', parse_dates=date_cols,
                         header = 0, sep = ',', engine= 'python'))
mces_lakes.head()




,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,...,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-04-16,...,NaN,1.00,Approved,m,NaN,0.156,Approved,mg/L,-92.971711,45.016556
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-01,...,NaN,NaN,NaN,m,NaN,NaN,NaN,mg/L,-92.971711,45.016556
2,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-02,...,NaN,0.66,Approved,m,NaN,0.107,Approved,mg/L,-92.971711,45.016556
3,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-16,...,NaN,0.66,Approved,m,NaN,0.141,Approved,mg/L,-92.971711,45.016556
4,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-30,...,NaN,0.50,Approved,m,NaN,0.029,Approved,mg/L,-92.971711,45.016556


In [5]:
mces_lakes.columns

Index(['PROJECT_ID', 'DATA_SET_TITLE', 'LAKE_NAME', 'CITY', 'COUNTY',
       'DNR_ID_Site_Number', 'MAJOR_WATERSHED', 'WATER_PLANNING_AUTHORITY',
       'LAKE_SITE_NUMBER', 'START_DATE', 'START_HOURMIN24', 'END_DATE',
       'END_HOURMIN24', 'SAMPLE_DEPTH_IN_METERS', 'Seasonal_Lake_Grade_RESULT',
       'Seasonal_Lake_Grade_QUALIFIER', 'Seasonal_Lake_Grade_Units',
       'Physical_Condition_RESULT', 'Physical_Condition_QUALIFIER',
       'Physical_Condition_Units', 'Recreational_Suitability_RESULT',
       'Recreational_Suitability_QUALIFIER', 'Recreational_Suitability_Units',
       'Secchi_Depth_RESULT_SIGN', 'Secchi_Depth_RESULT',
       'Secchi_Depth_QUALIFIER', 'Secchi_Depth_Units',
       'Total_Phosphorus_RESULT_SIGN', 'Total_Phosphorus_RESULT',
       'Total_Phosphorus_QUALIFIER', 'Total_Phosphorus_Units', 'longitude',
       'latitude'],
      dtype='object')

In [14]:
from dfply import *
from more_dfply import recode, ifelse
lakes_w_complete_measurements =(mces_lakes
                                >> select(X.LAKE_NAME, X.DNR_ID_Site_Number, X.START_DATE, X.Secchi_Depth_RESULT, X.Total_Phosphorus_RESULT)
                                >> filter_by(pd.notna(mces_lakes.Total_Phosphorus_RESULT)& pd.notna(mces_lakes.Secchi_Depth_RESULT))
                                >> mutate(Year = X.START_DATE.dt.year)
                                >> drop(X.START_DATE)
                                >> filter_by(X.Year >=2004))

In [16]:
lakes_w_complete_measurements.head()

,LAKE_NAME,DNR_ID_Site_Number,Secchi_Depth_RESULT,Total_Phosphorus_RESULT,Year
0,Acorn Lake,82010200-01,1.00,0.156,2006
2,Acorn Lake,82010200-01,0.66,0.107,2006
3,Acorn Lake,82010200-01,0.66,0.141,2006
4,Acorn Lake,82010200-01,0.50,0.029,2006
5,Acorn Lake,82010200-01,0.50,0.058,2006


In [28]:
lakes_to_keep= (lakes_w_complete_measurements
                >> group_by(X.DNR_ID_Site_Number, X.LAKE_NAME, X.Year)
                >> summarise(cnt= n(X.DNR_ID_Site_Number))
                >> ungroup
                >> group_by(X.DNR_ID_Site_Number, X.LAKE_NAME)
                >> summarise(ct= n(X.DNR_ID_Site_Number))
                >> filter_by(X.ct >=11))

In [29]:
lakes_to_keep.shape

(49, 3)

In [30]:
lakes_to_keep.head()

,LAKE_NAME,DNR_ID_Site_Number,ct
1,George Watch Lake,02000500-01,11
16,Riley Lake,10000200-01,11
23,St. Joe Lake,10001100-01,11
29,Bavaria Lake,10001900-01,11
34,Reitz Lake,10005200-01,11


In [35]:
lake_year_stats = (lakes_w_complete_measurements
                   >> filter_by(X.DNR_ID_Site_Number.isin(lakes_to_keep.DNR_ID_Site_Number))
                   >> group_by(X.DNR_ID_Site_Number, X.LAKE_NAME, X.Year)
                   >> summarise(mean_phos = X.Total_Phosphorus_RESULT.mean(),
                                med_phos = X.Total_Phosphorus_RESULT.median(),
                                sd_phos = X.Total_Phosphorus_RESULT.std(),
                                mean_secci = X.Secchi_Depth_RESULT.mean(),
                                med_secci = X.Secchi_Depth_RESULT.median(),
                                sd_secci = X.Secchi_Depth_RESULT.std()))

In [37]:
lake_year_stats.to_csv("./MinneMUDAC_raw_files/lake_year_stats.csv")